In [302]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
from torch import tensor
from torch.functional import F
from torchvision import transforms, datasets
from torchvision import models
from torch.utils.data.sampler import WeightedRandomSampler
from torch.nn.utils.rnn import pad_sequence


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, root_mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

from skimage.morphology import skeletonize
from itertools import product

import pandas as pd
import numpy as np
import os
from PIL import Image
import copy
import cv2
import time
import shutil
import random
import h5py
from Helpers import Params, collate_fn

from matplotlib import pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [303]:
class ProgressBar:
    def __init__(self, total: int):
        self.total = total
        self.current = 0
        self.last_progress = 0
        self.start_time = time.time()

    def update(self, current: int, epochs: str):
        self.current = current
        progress = (self.current / self.total) * 100
        if int(progress) > self.last_progress:
            elapsed_time = time.time() - self.start_time
            print(
                f'\rEpoch: {epochs.rjust(7)} {str(int(progress)).rjust(3)}% | Elapsed: {str(int(elapsed_time)).rjust(3)}s', end='')
            self.last_progress = int(progress)

In [304]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, path='checkpoint.pt', verbose=False):
        self.patience = patience
        self.delta = delta
        self.path = path
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(
                    f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(
                f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [305]:
class TrainingHistory:
    def __init__(self):
        self.history = {
            'train_loss': [],
            'train_accuracy': [],
            'val_loss': [],
            'val_accuracy': []
        }

    def update(self, train_loss: float, train_accuracy: float, val_loss: float, val_accuracy: float):
        self.history['train_loss'].append(train_loss)
        self.history['train_accuracy'].append(train_accuracy)
        self.history['val_loss'].append(val_loss)
        self.history['val_accuracy'].append(val_accuracy)

    def get_history(self):
        return self.history

    def plot(self):
        epochs = range(1, len(self.history['train_loss']) + 1)
        plt.figure(figsize=(12, 6))

        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.history['train_loss'], label='Train Loss')
        plt.plot(epochs, self.history['val_loss'], label='Validation Loss')
        plt.title('Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        # Plot accuracy
        plt.subplot(1, 2, 2)
        plt.plot(
            epochs, self.history['train_accuracy'], label='Train Accuracy')
        plt.plot(epochs, self.history['val_accuracy'],
                 label='Validation Accuracy')
        plt.title('Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()

In [306]:
class TrainingHistory:
    def __init__(self):
        self.history = {
            'train_loss': [],
            'train_accuracy': [],
            'val_loss': [],
            'val_accuracy': []
        }

    def update(self, train_loss: float, train_accuracy: float, val_loss: float, val_accuracy: float):
        self.history['train_loss'].append(train_loss)
        self.history['train_accuracy'].append(train_accuracy)
        self.history['val_loss'].append(val_loss)
        self.history['val_accuracy'].append(val_accuracy)

    def get_history(self):
        return self.history

    def plot(self):
        epochs = range(1, len(self.history['train_loss']) + 1)
        plt.figure(figsize=(12, 6))

        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.history['train_loss'], label='Train Loss')
        plt.plot(epochs, self.history['val_loss'], label='Validation Loss')
        plt.title('Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        # Plot accuracy
        plt.subplot(1, 2, 2)
        plt.plot(
            epochs, self.history['train_accuracy'], label='Train Accuracy')
        plt.plot(epochs, self.history['val_accuracy'],
                 label='Validation Accuracy')
        plt.title('Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.tight_layout()
        plt.show()

In [307]:
class MetricsTracker:
    def __init__(self):
        self.losses = []
        self.accuracies = []

    def update(self, loss: float, accuracy: float):
        self.losses.append(loss)
        self.accuracies.append(accuracy)

    def get_average_loss(self):
        return sum(self.losses) / len(self.losses) if self.losses else 0.0

    def get_average_accuracy(self):
        return sum(self.accuracies) / len(self.accuracies) if self.accuracies else 0.0

In [308]:
class Validator:
    def __init__(self, model: nn.Module, criterion: nn.CrossEntropyLoss, device: torch.device):
        self.model = model
        self.criterion = criterion
        self.device = device

    def validate(self, val_loader: DataLoader) -> tuple:
        self.model.eval()
        metrics_tracker = MetricsTracker()

        with torch.no_grad():
            for image, midi in val_loader:
                image: torch.Tensor = image.to(self.device)
                midi: torch.Tensor = midi.to(self.device)

                input_tokens: torch.Tensor = midi[:, :-1]
                target_tokens: torch.Tensor = midi[:, 1:]
                target_tokens = target_tokens.reshape(-1)

                output: torch.Tensor = self.model(image, input_tokens)
                output = output.reshape(-1, output.shape[-1])

                loss: torch.Tensor = self.criterion(output, target_tokens)
                accuracy = (output.argmax(dim=1) ==
                            target_tokens).float().mean().item()

                metrics_tracker.update(loss.item(), accuracy)

        avg_loss = metrics_tracker.get_average_loss()
        avg_accuracy = metrics_tracker.get_average_accuracy()

        return avg_loss, avg_accuracy

In [309]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model).to(device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(
            0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor):
        x = x + self.pe[:, :x.size(1)]
        return x

In [310]:
class ValidatorWithOutputs:
    def __init__(self, model: nn.Module, criterion: nn.CrossEntropyLoss, device: torch.device):
        self.model = model
        self.criterion = criterion
        self.device = device

    def validate(self, val_loader: DataLoader) -> tuple:
        self.model.eval()
        outputs = []

        with torch.no_grad():
            for image, midi in val_loader:
                image: torch.Tensor = image.to(self.device)
                midi: torch.Tensor = midi.to(self.device)

                input_tokens: torch.Tensor = midi[:, :-1]
                target_tokens: torch.Tensor = midi[:, 1:]
                target_tokens = target_tokens.reshape(-1)

                output: torch.Tensor = self.model(image, input_tokens)
                output = output.reshape(-1, output.shape[-1])

                predicted = output.argmax(dim=1).cpu().tolist()
                expected = target_tokens.cpu().tolist()

                outputs.append((predicted, expected))

        return outputs

In [311]:
def train(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    criterion: nn.CrossEntropyLoss,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device = device,
    history: TrainingHistory = None,
    early_stopping: EarlyStopping = None
):
    for epoch in range(epochs):
        progress_bar = ProgressBar(len(train_loader))
        metrics_tracker = MetricsTracker()
        validator = Validator(model, criterion, device)
        model.train()

        for i, (image, word) in enumerate(train_loader):
            image: torch.Tensor = image.to(device)
            word: torch.Tensor = word.to(device)

            input_tokens = word[:, :-1]
            target_tokens = word[:, 1:]
            # print(input_tokens)
            # print(target_tokens)
            target_tokens = target_tokens.reshape(-1)

            output: torch.Tensor = model(image, input_tokens)
            output = output.reshape(-1, output.shape[-1])

            loss: torch.Tensor = criterion(output, target_tokens)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            accuracy = (output.argmax(dim=1) == target_tokens).float().mean().item()
            metrics_tracker.update(loss.item(), accuracy)

            progress_bar.update(i + 1, f'{epoch + 1}/{epochs}')

        print(
            f' | loss: {metrics_tracker.get_average_loss():.4f} - acc: {metrics_tracker.get_average_accuracy():.4f}', end='')
        val_loss, val_acc = validator.validate(val_loader)
        print(f' | val_loss: {val_loss:.4f} - val_acc: {val_acc:.4f}')

        if history:
            history.update(metrics_tracker.get_average_loss(
            ), metrics_tracker.get_average_accuracy(), val_loss, val_acc)

        if early_stopping:
            early_stopping(val_loss, model)
            if early_stopping.early_stop:
                print("Early stopping")
                break

In [312]:
class HandWritingDataset(datasets.ImageFolder):
    def __init__(
        self,
        root,
        transform: transforms.Compose = None,
        label_transofrm=None,
        augument: transforms.Compose = None
    ):
        if augument is not None:
            transform = transforms.Compose([
                augument,
                transform
            ])

        super(HandWritingDataset, self).__init__(root, transform=transform)
        self.classes = [''.join([i if i != '_' else '' for i in word])
                        for word in self.classes]
        self.label_transform = label_transofrm

    def __getitem__(self, index):
        image, label = super(HandWritingDataset, self).__getitem__(index)
        if self.label_transform is not None:
            label = tensor(self.label_transform(self.classes[label]))
        label = torch.cat((tensor([27]), label, tensor([28])))
        return image, label

    def create_sampler(self, epsilon: float = 0.004):
        """
        epsilon: how strongly flatten the distribution? 0 means flat, the greater the more similar to original distribution, after 0.01 there is little difference
        """
        df = pd.DataFrame(self.samples, columns=['filename', 'class'])
        df['label'] = df['class'].apply(lambda x: ''.join(
            [i for i in self.classes[x] if i != ' ']))
        df['length'] = df['label'].apply(len)
        length_df = df['length'].value_counts().sort_index().to_dict()
        class_count = df['class'].value_counts().sort_index()
        df['class_length'] = df['length'].map(length_df)
        df['class_count'] = df['class'].apply(lambda x: class_count.iloc[x])
        df['result'] = 1.0 / df['class_length'] + epsilon
        return WeightedRandomSampler(df['result'].values, len(df['result']))

In [313]:
class H5Dataset:
    def __init__(self, file_path: str, num_epochs: int):
        """
        Args:
            num_epochs (int): Number of epochs to split the dataset into.
        """
        self.file_path = file_path
        with h5py.File(file_path, 'r') as h5f:
            h5_size = len(h5f['images'])
            self.num_epochs = num_epochs
            self.epoch_size = h5_size // num_epochs
            self.current_epoch = 0
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
        ])

    def __len__(self):
        return self.epoch_size

    def __getitem__(self, index):
        with h5py.File(self.file_path, 'r') as h5f:
            image = h5f['images'][self.current_epoch * self.epoch_size + index]
            image = self.transform(image)
            image = image.permute(1, 2, 0)
            image = torch.stack([image[0]] * 3, dim=0)

            label = h5f['labels'][self.current_epoch * self.epoch_size + index]
            label = label[:np.argmax(label == 0)] if 0 in label else label
            label = torch.tensor(label, dtype=torch.long)
            label = torch.cat((tensor([27]), label, tensor([28])))

        return image, label

    def next_epoch(self):
        self.current_epoch = (self.current_epoch + 1) % self.num_epochs

    def create_h5_sampler(self, epsilon: float = 0.004):
        """
        epsilon: how strongly flatten the distribution? 0 means flat, the greater the more similar to original distribution, after 0.01 there is little difference
        """
        df = pd.DataFrame({'index': range(
            self.current_epoch * self.epoch_size, (self.current_epoch + 1) * self.epoch_size)})
        with h5py.File(self.file_path, 'r') as h5f:
            labels = h5f['labels'][self.current_epoch *
                                   self.epoch_size: (self.current_epoch + 1) * self.epoch_size]
            df['length'] = (labels != 0).sum(axis=1)

        length_df = df['length'].value_counts().sort_index()
        length_dict = length_df.to_dict()

        df['class_length'] = df['length'].map(length_dict)
        df['result'] = 1.0 / df['class_length'] + epsilon

        return WeightedRandomSampler(df['result'].values, len(df['result']))

In [314]:
class HandwritingTransformer(nn.Module):
    def __init__(
        self,
        input_size: int,
        vocab_size: int,
        d_model: int = 128,
        nhead_en: int = 1,
        num_layers_en: int = 1,
        nhead_de: int = 1,
        num_layers_de: int = 1,
        dropout: float = 0.2  # Added dropout parameter
    ):
        super(HandwritingTransformer, self).__init__()

        self.cnn = models.mobilenet_v2(pretrained=True).features[:4]
        for param in self.cnn.parameters():
            param.requires_grad = False

        self.input_size = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, 200)

        # Add LayerNorm and Dropout after input_size
        self.input_norm = nn.LayerNorm(d_model)
        self.input_dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model, nhead_en, dropout=dropout, norm_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers_en)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model, nhead_de, dropout=dropout, norm_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers_de)

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_decoder = PositionalEncoding(d_model, 20)

        # Add LayerNorm and Dropout after embedding
        self.embedding_norm = nn.LayerNorm(d_model)
        self.embedding_dropout = nn.Dropout(0.5)

        self.output = nn.Linear(d_model, vocab_size)

    def forward(self, src: torch.Tensor, tgt: torch.Tensor):
        src = torch.stack([src[:, 0]] * 3, dim=1)
        src = self.cnn(src)

        src = src.flatten(1, 2)
        src = src.permute(0, 2, 1)
        src = self.input_size(src)

        # Apply LayerNorm and Dropout after input_size
        src = self.input_norm(src)
        src = self.input_dropout(src)

        src = self.pos_encoder(src)
        src = src.permute(1, 0, 2)
        memory = self.encoder(src)

        tgt = self.embedding(tgt)

        # Apply LayerNorm and Dropout after embedding
        tgt = self.embedding_norm(tgt)
        tgt = self.embedding_dropout(tgt)

        tgt = self.pos_decoder(tgt)
        tgt = tgt.permute(1, 0, 2)

        tgt_mask = self.generate_square_subsequent_mask(
            tgt.size(0)).to(tgt.device)

        output: torch.Tensor = self.decoder(tgt, memory, tgt_mask=tgt_mask)
        output = self.output(output)
        output = output.permute(1, 0, 2)
        return output

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

In [315]:
augument = transforms.Compose([
    # transforms.RandomRotation(15, expand=True, fill=(255,)),
    transforms.RandomAffine(0, translate=(0.05, 0.05), fill=(255,)),
    # transforms.RandomPerspective(distortion_scale=0.5, p=0.5, fill=(255,)),
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
])

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((64, 128)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: 1.0 - x),
    transforms.Normalize((0.5,), (0.5,)),
])

In [317]:
# train_dataset = HandWritingDataset('words_data/train', transform=transform, label_transofrm=Params.encode_string, augument=augument) # augument data
train_dataset = H5Dataset('train_data.h5', num_epochs=1)
val_dataset = HandWritingDataset(root='words_data/val', transform=transform, label_transofrm=Params.encode_string)

train_loader = DataLoader(train_dataset, batch_size=64, num_workers=4, sampler=train_dataset.create_h5_sampler(0), collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=4, shuffle=True, collate_fn=collate_fn)

model = HandwritingTransformer(
    input_size=16 * 24,
    vocab_size=len(Params.vocab)+2,
    d_model=128,
    nhead_en=1,
    num_layers_en=1,
    nhead_de=1,
    num_layers_de=1,
    dropout=0.2
).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
early_stopping = EarlyStopping(patience=5)
history = TrainingHistory()

/home/jakub/miniconda3/envs/torch/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jakub/miniconda3/envs/torch/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/jakub/miniconda3/envs/torch/lib/python3.13/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [318]:
train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=50,
    early_stopping=early_stopping,
    history=history
)

Epoch:    1/50 100% | Elapsed: 237s | loss: 1.3753 - acc: 0.3237 | val_loss: 0.8159 - val_acc: 0.3374
Epoch:    2/50 100% | Elapsed: 266s | loss: 0.9848 - acc: 0.3938 | val_loss: 0.6351 - val_acc: 0.3631
Epoch:    3/50 100% | Elapsed: 273s | loss: 0.8736 - acc: 0.4147 | val_loss: 0.6051 - val_acc: 0.3753
Epoch:    4/50   1% | Elapsed:   3s

KeyboardInterrupt: 

In [319]:
test_dataset = HandWritingDataset(root='words_data/val', transform=transform, label_transofrm=Params.encode_string)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=4, shuffle=True, collate_fn=collate_fn)

val_with_outs = ValidatorWithOutputs(model, criterion, device)
outputs = val_with_outs.validate(test_loader)

for i, (predicted, expected) in enumerate(outputs):
    if i > 100:
        break
    predicted = Params.decode_string([i for i in predicted if i not in [27, 28]])
    expected = Params.decode_string([i for i in expected if i not in [27, 28]])
    print(f"{predicted.ljust(12)}  -  {expected.ljust(12)}")

wven          -  even        
tir           -  her         
hese          -  have        
oe            -  be          
it            -  if          
the           -  the         
staongl       -  straight    
of            -  of          
aitk          -  sick        
ot            -  at          
especiably    -  especially  
to            -  to          
through       -  through     
on            -  on          
molleon       -  million     
unioorm       -  uniform     
as            -  is          
enay          -  away        
oboolutely    -  absolutely  
eggur         -  enjoy       
up            -  up          
a             -  a           
to            -  to          
oll           -  all         
hheught       -  though      
fre           -  free        
is            -  in          
of            -  of          
enderpinning  -  underpinning
wodds         -  moods       
planwd        -  played      
tor           -  for         
one           -  one         
growing   